In [1]:
using Revise
using DifferenceEquations
using Distributions
using Optim
using Random

# State space Models 

## ARMA(p,q)

Define classical univariate ARMA(p,q) model (zero mean): 

$$
\begin{aligned}
y_t = \sum_{i=1}^{r} \phi_i y_{t-i} + \sum_{j=0}^{r} \theta_{j}\epsilon_{t-j} 
\end{aligned}
$$
where $r=max(p,q+1)$ and i,j s.t. $p<i<r$ or $ q<i<r$ $\phi_i=0, \theta_j=0$.
Using result from Hamilton, we can write it the model in teh state space.

Transition matrix:
$$
\begin{aligned}
\zeta_{t+1} = \left( \begin{matrix}
\phi_1 &\phi_2 &... &\phi_{r-1}&\phi_r\\
1&0 &...&0& 0\\
\cdots&\cdots &...&\cdots&\cdots\\
0&0&...&1&0
\end{matrix}\right) \zeta_{t} + \left[ \begin{matrix} \epsilon_t\\ 0\\\cdots \\ 0\end{matrix}\right]
\end{aligned}
$$
Observation matrix:
$$
\begin{aligned}
y_t = [1, \theta_1, ... \theta_{r-1}]\zeta_{t}
\end{aligned}
$$


### Implementation:

In [2]:
#r is a size of r= max(p, q+1)
function ARMA_transition(u, p, t)
    r = length(u)
    matrix_T = zeros(r,r)
    
    for i in 1:r
        matrix_T[1,i] = p[i]
    end
    
    for i in 1:r
        for j in 2:r
            if j==i && j<r
                 matrix_T[j,i] = 1.0
            end
        end
    end
    
    return matrix_T*u
end

function ARMA_noise(u, p, t, noise) # g
    r = length(u)
    z = zeros(r)
    z[1] = p[2r+1]*randn()
    return z
end

function ARMA_observation(u, p, t, noise) # g
    r = length(u)
    vec_MA = zeros(r)
    for i in 1:r
        vec_MA[i] = p[r+i]
    end
    return sum(vec_MA'*u)
end


ARMA_observation (generic function with 1 method)

Rest of values for the `state_space` struct

In [3]:


p_1=5
q =4
ρ = 0.95
θ = 0.7
σ = 0.02
r = max(p_1,q+1)

p = zeros(2*r+1)
for i in 1:p_1
        p[i] = ρ^i
end

for i in 1:q
        p[r+i] = θ^i
end
p[2*r+1] = σ


tspan = (1, 50)

(1, 50)

#### Simple data generation 


In [4]:
ζ = zeros(r,tspan[2]+r)
n = σ .* randn(size(ζ,2))
ζ[1,:] += n
Y = 
for t in 2:tspan[2]
    ζ[:,t] += ARMA_transition(ζ[:,t-1],p,t)
end
Y = zeros(1,tspan[2])
for t in 1:tspan[2]
    Y[t] = ARMA_observation(ζ[:,t-1+r],p,t, nothing)
end
u0 = ζ[:,r]


5-element Vector{Float64}:
 -0.019638471335738233
  0.0
  0.0
  0.0
  0.0

### Definition in `StateSpaceProblem`

In [5]:
prob = StateSpaceProblem(
    ARMA_transition, 
    ARMA_noise, 
    ARMA_observation, 
    MvNormal(0,1),
    tspan,
    u0,
    noise = [0.0, 0.0],
    observables= Y
)

StateSpaceProblem{Val{false}(), typeof(ARMA_transition), typeof(ARMA_noise), typeof(ARMA_observation), Vector{Float64}, DifferenceEquations.Gaussian{Int64, Nothing}, ZeroMeanIsoNormal{Tuple{Base.OneTo{Int64}}}, Tuple{Int64, Int64}, Matrix{Float64}, Vector{Float64}}(ARMA_transition, ARMA_noise, ARMA_observation, [0.0, 0.0], DifferenceEquations.Gaussian{Int64, Nothing}(0, nothing), ZeroMeanIsoNormal(
dim: 0
μ: 0-element Zeros{Float64}
Σ: Matrix{Float64}(undef, 0, 0)
)
, (1, 50), [-0.013746929935016762 -0.0028542150659986447 … 0.0152890286748277 -0.0047194348313228975], [-0.019638471335738233, 0.0, 0.0, 0.0, 0.0])

## Nicholson and Bailey (1935)

Here is a classical model (at least what I got for goggle) from ecology (population of some genre and it's parasite):
 
Let $H_t$ be a (biomass) population of some animal (host), $P_t$ population of it's parasite. 

$$
\begin{aligned}
H_{t+1} = \lambda H_t \exp(-a P_t)\\
P_{t+1} = H_t (1 - \exp(-a P_t)
\end{aligned}
$$
I add some random noises:
$$
\begin{aligned}
H_{t+1} =\lambda H_t \exp(-a P_t)+z_{1t}\\
P_{t+1} = H_t (1 - \exp(-a P_t)+z_{2t}
\end{aligned}
$$
where

$$
\begin{aligned}
z_{1t} \sim N(0,\sigma_1)\\
z_{2t} \sim N(0,\sigma_2)
\end{aligned}
$$
I assume we just observe the population of the animal, and starting population of the paraiste

Link: https://en.wikipedia.org/wiki/Nicholson%E2%80%93Bailey_modelhttps://en.wikipedia.org/wiki/Nicholson%E2%80%93Bailey_model

### Code implementation

In [6]:
#f
function NB_transition(u, p, t)
    u_prim = zeros(2)
    u_prim[1] = p[1]*u[1]*exp(-p[2]*u[2])
    u_prim[2] = u[1]*(1-exp(-p[2]*u[2]))
    return u_prim
end
#g
function NB_noise(u, p, t, noise) # g
    z1 = p[3] * randn()
    z2 = p[4]*randn()
    return [z1; z2]
end

#h
function NB_observation(u, p, t, noise) # h
    return [1 0] * u
end



NB_observation (generic function with 1 method)

Rest of values of `state_space` struct

In [7]:

tspan = (1, 15)

#parameters
lambda = 1.02
a = 1
sigma_1 = 0.005
sigma_2 = 0.0001

p = [lambda a sigma_1 sigma_2]
#u0
u0 = [1 0.1]



1×2 Matrix{Float64}:
 1.0  0.1

#### Simple data generation 
I haven't checked the "good" params, so here it might be pretty random 

In [8]:
Y = zeros(2, tspan[2]) 
n1 = sigma_1 .* randn(size(Y,2))
n2 = sigma_2 .* randn(size(Y,2))
Y[1,:] += n1
Y[2,:] += n2
Y[:,1] =u0

for t in 2:size(Y,2)
    Y[:,t] += NB_transition(Y[:,t-1], p, t)
end
observables = Y[1,:]

15-element Vector{Float64}:
 1.0
 0.9233571051182355
 0.8571170001917893
 0.8009273436504152
 0.7660225722157694
 0.7407626638945856
 0.7276186147606544
 0.7220559077505675
 0.7186359992960852
 0.7228265712893535
 0.7314405867290603
 0.7369394038076666
 0.7458250899560609
 0.7657115935830267
 0.7805542164635214

### Definition in `StateSpaceProblem`

In [10]:
prob = StateSpaceProblem(
    NB_transition, 
    NB_noise, 
    NB_observation, 
    MvNormal(0,1),
    tspan,
    u0,
    observables= observables
)

StateSpaceProblem{Val{false}(), typeof(NB_transition), typeof(NB_noise), typeof(NB_observation), DifferenceEquations.Gaussian{Int64, Nothing}, DifferenceEquations.Gaussian{Int64, Nothing}, ZeroMeanIsoNormal{Tuple{Base.OneTo{Int64}}}, Tuple{Int64, Int64}, Vector{Float64}, Matrix{Float64}}(NB_transition, NB_noise, NB_observation, DifferenceEquations.Gaussian{Int64, Nothing}(0, nothing), DifferenceEquations.Gaussian{Int64, Nothing}(0, nothing), ZeroMeanIsoNormal(
dim: 0
μ: 0-element Zeros{Float64}
Σ: Matrix{Float64}(undef, 0, 0)
)
, (1, 15), [1.0, 0.9233571051182355, 0.8571170001917893, 0.8009273436504152, 0.7660225722157694, 0.7407626638945856, 0.7276186147606544, 0.7220559077505675, 0.7186359992960852, 0.7228265712893535, 0.7314405867290603, 0.7369394038076666, 0.7458250899560609, 0.7657115935830267, 0.7805542164635214], [1.0 0.1])